# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from scipy.stats import linregress
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [19]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500) 
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500) 
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

42

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [33]:
#build the dataframe
city_data_df = pd.DataFrame(columns=(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 
                                      'Wind Speed', 'Country', 'Date']))
#load cities in df
city_data_df['City'] = cities
failed = 0

base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

for index, row in city_data_df.iterrows():
    city = row['City']
    try:
        query_url = f"{base_url}q={city}&units={units}&appid={weather_api_key}"  
        response = requests.get(query_url).json() 
        #load city weather data into data frame
        city_data_df.loc[index, 'City'] = response['name']
        city_data_df.loc[index, 'Lat'] = response['coord']['lat']
        city_data_df.loc[index, 'Lng'] = response['coord']['lon']
        city_data_df.loc[index, 'Max Temp'] = response['main']['temp_max'] ### Farenheit ###
        city_data_df.loc[index, 'Humidity'] = response['main']['humidity'] ### percentage ###
        city_data_df.loc[index, 'Cloudiness'] = response['clouds']['all'] ### percentage ###
        city_data_df.loc[index, 'Wind Speed'] = response['wind']['speed'] ### mph ###
        city_data_df.loc[index, 'Country'] = response['sys']['country']
        city_data_df.loc[index, 'Date'] = response['dt']
        print(f"Processing record #{index} of city: {row['City']}")
    except KeyError:
        print(f"Missing field/result for record #{index} of city: {row['City']}")
        city_data_df.drop(index, inplace=True)
        failed += 1

print(f"Total of {failed} records dropped.")

Processing record #0 of city: Te Anau
Processing record #1 of city: San Patricio
Processing record #2 of city: Dingle
Processing record #3 of city: Huacho
Processing record #4 of city: Busselton
Processing record #5 of city: Hilo
Processing record #6 of city: Rikitea
Missing field/result for record #7 of city: barentsburg
Processing record #8 of city: zonguldak
Processing record #9 of city: bathsheba
Processing record #10 of city: atuona
Processing record #11 of city: tasiilaq
Processing record #12 of city: victoria
Processing record #13 of city: bethel
Processing record #14 of city: butaritari
Processing record #15 of city: saldanha
Processing record #16 of city: mataura
Processing record #17 of city: qaanaaq
Processing record #18 of city: cape town
Processing record #19 of city: jamestown
Processing record #20 of city: torbay
Processing record #21 of city: khatanga
Processing record #22 of city: puerto narino
Processing record #23 of city: puerto ayora
Processing record #24 of city: 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [34]:
city_data_df.to_csv(output_data_file, index=False)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Te Anau,-45.4167,167.717,55.8,76,90,7.85,NZ,1618795382
1,San Patricio,28.017,-97.5169,62.01,77,90,7.49,US,1618795383
2,Dingle,10.9995,122.671,81.21,69,100,3.78,PH,1618795383
3,Huacho,-11.1067,-77.605,67.93,85,28,6.6,PE,1618795383
4,Busselton,-33.65,115.333,63,80,100,1.01,AU,1618795384


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [35]:
over_humid = city_data_df.loc[city_data_df['Humidity'] > 100]
over_humid

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


In [36]:
#  Get the indices of cities that have humidity over 100%.
#  N/A

In [37]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
# N/A

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

In [38]:
#read in cities.csv file to data frame
weather_df = pd.read_csv("output_data/cities.csv")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Te Anau,-45.4167,167.7167,55.80,76,90,7.85,NZ,1618795382
1,San Patricio,28.0170,-97.5169,62.01,77,90,7.49,US,1618795383
2,Dingle,10.9995,122.6711,81.21,69,100,3.78,PH,1618795383
3,Huacho,-11.1067,-77.6050,67.93,85,28,6.60,PE,1618795383
4,Busselton,-33.6500,115.3333,63.00,80,100,1.01,AU,1618795384


## Latitude vs. Temperature Plot

In [ ]:
#DataFrame.plot.scatter(x, y, s=None, c=None, **kwargs)


#plt.title("Capomulin Tumor Volume vs. Weight")
#plt.xlabel("Mouse Weight")
#plt.ylabel("Average Tumor Volume (mm3)")
#plt.scatter(mouse_weight, avg_tumor_vol, marker='o', color='green')

#plt.savefig('plots/ScatterPlot')
#plt.show()

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression